In [43]:
!pip install faker
!pip install psycopg2-binary
import requests
import json
from faker import Faker
import random
import psycopg2
from psycopg2 import sql
from faker import Faker
import random
import psycopg2

def cria_conectores():
    postgres = {
        "config": {
            "connector.class": "io.debezium.connector.postgresql.PostgresConnector",
            "database.dbname": "db",
            "database.hostname": "postgres",
            "database.password": "admin",
            "database.port": "5432",
            "database.server.name": "postgres",
            "database.user": "user",
            "delete.handling.mode": "rewrite",
            "plugin.name": "pgoutput",
            "table.include.list": "public.condominios, public.imoveis,public.moradores,public.transacoes",
            "table.whitelist": "public.condominios, public.imoveis,public.moradores,public.transacoes",
            "topic.prefix": "postgres"
        },
        "name": "postgres-source-connector"
    }
    requests.post('http://connect:8083/connectors', json=postgres)
    
    minio = {
        "name": "minio-sink-connector",
        "config": {
            "connector.class": "io.aiven.kafka.connect.s3.AivenKafkaConnectS3SinkConnector",
            "aws.access.key.id": "SmRXzZ0KrpROvPSKDddy",
            "aws.secret.access.key": "ApanXjLP51CxV3bjPqOlankgLDpeoIZyR4xewaOS",
            "aws.s3.bucket.name": "condomanage",
            "aws.s3.endpoint": "http://minio:9000",
            "aws.s3.region": "us-east-1",
            "format.output.type": "parquet",
            "topics": "postgres.public.condominios, postgres.public.imoveis, postgres.public.moradores, postgres.public.transacoes",
            "file.compression.type": "none",
            "flush.size": "20",
            "file.name.template": "raw/cdc/{{topic}}/{{timestamp:unit=yyyy}}{{timestamp:unit=MM}}{{timestamp:unit=dd}}_{{partition:padding=true}}-{{start_offset:padding=true}}.parquet"
        }
    }
    
    requests.post('http://connect:8083/connectors', json=minio)




fake = Faker()

# Função para conectar ao banco de dados
def connect_to_db():
    return psycopg2.connect(
        dbname='db',
        user='user',
        password='admin',
        host='postgres',
        port='5432'
    )

# Função para executar comandos SQL
def execute_sql_commands(commands):
    conn = connect_to_db()
    try:
        with conn.cursor() as cursor:
            for command in commands:
                cursor.execute(command)
        conn.commit()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

# Inserir condomínios e recuperar IDs
def insert_condominios():
    conn = connect_to_db()
    condominios = []
    condominio_map = {}
    try:
        with conn.cursor() as cursor:
            for i in range(1, 101):
                nome = fake.company()
                endereco = fake.address().replace('\n', ', ')
                cursor.execute("INSERT INTO condominios (nome, endereco) VALUES (%s, %s) RETURNING condominio_id;", (nome, endereco))
                condominio_id = cursor.fetchone()[0]
                condominio_map[condominio_id] = (nome, endereco)
                condominios.append(condominio_id)
            conn.commit()
    except Exception as e:
        print(f"An error occurred while inserting condominios: {e}")
    finally:
        conn.close()
    return condominios

# Inserir moradores, imóveis e transações
def insert_related_data(condominio_ids):
    conn = connect_to_db()
    moradores = []
    imoveis = []
    transacoes = []
    try:
        with conn.cursor() as cursor:
            # Inserir moradores
            for i in range(1, 101):
                nome = fake.name()
                condominio_id = random.choice(condominio_ids)
                data_registro = fake.date_this_decade()
                cursor.execute("INSERT INTO moradores (nome, condominio_id, data_registro) VALUES (%s, %s, %s) RETURNING morador_id;", (nome, condominio_id, data_registro))
                morador_id = cursor.fetchone()[0]
                moradores.append(morador_id)

            # Inserir imóveis
            for i in range(1, 101):
                tipo = random.choice(['Apartamento', 'Casa'])
                condominio_id = random.choice(condominio_ids)
                valor = round(random.uniform(100000, 1000000), 2)
                cursor.execute("INSERT INTO imoveis (tipo, condominio_id, valor) VALUES (%s, %s, %s) RETURNING imovel_id;", (tipo, condominio_id, valor))
                imovel_id = cursor.fetchone()[0]
                imoveis.append(imovel_id)

            # Inserir transações
            for i in range(1, 101):
                imovel_id = random.choice(imoveis)
                morador_id = random.choice(moradores)
                data_transacao = fake.date_this_year()
                valor_transacao = round(random.uniform(50000, 500000), 2)
                cursor.execute("INSERT INTO transacoes (imovel_id, morador_id, data_transacao, valor_transacao) VALUES (%s, %s, %s, %s);", (imovel_id, morador_id, data_transacao, valor_transacao))

            conn.commit()
    except Exception as e:
        print(f"An error occurred while inserting related data: {e}")
    finally:
        conn.close()

# Executar o processo
condominio_ids = insert_condominios()
insert_related_data(condominio_ids)

In [1]:
#INGESTÃO NO DATA LAKE
import os
import pyspark
from pyspark.sql import SparkSession

## DEFINE SENSITIVE VARIABLES
WAREHOUSE = os.environ.get("WAREHOUSE") 
AWS_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY_ID") 
AWS_SECRET_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY") 
AWS_S3_ENDPOINT = os.environ.get("AWS_S3_ENDPOINT") ## MINIO ENDPOINT
PG_USER = os.environ.get("PG_USER")
PG_PASSWORD = os.environ.get("PG_PASSWORD")
PG_DB = os.environ.get("PG_DB")

conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-core_2.12:2.4.0,org.postgresql:postgresql:42.6.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
        .set('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
        .set('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
        .set("spark.hadoop.fs.s3a.path.style.access", "true")
        .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .set("spark.hadoop.fs.s3a..connection.ssl.enabled", "true")
        
        .set("spark.hadoop.fs.s3a.endpoint", AWS_S3_ENDPOINT)
        .set('spark.hadoop.fs.s3a.access.key', AWS_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', AWS_SECRET_KEY)
)

# Inicializa a sessão Spark com suporte ao Delta Lake
spark = SparkSession.builder \
    .appName("DataIngestion") \
    .master("local")\
    .config(conf=conf)\
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

# Configurações do PostgreSQL
pg_url = "jdbc:postgresql://postgres:5432/" + PG_DB
pg_properties = {
    "user": PG_USER,
    "password": PG_PASSWORD,
    "driver": "org.postgresql.Driver"
}

# Carrega dados das tabelas PostgreSQL
def load_table(table_name):
    return spark.read.jdbc(url=pg_url, table=table_name, properties=pg_properties)

# Salva os dados no MinIO em formato Parquet
def save_to_minio(df, path):
    df.write.mode("overwrite").parquet(f"{WAREHOUSE}/raw/full/{path}")

tables = ('condominios', 'moradores', 'transacoes', 'imoveis')

for table in tables:
    df = load_table(f"public.{table}")
    save_to_minio(df, table)

spark.stop()

In [13]:
# BRONZE
import os
import pyspark
from pyspark.sql import SparkSession, HiveContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col, expr, to_date
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

import pandas as pd
import struct

def bytes_to_decimal(byte_list):
    if not byte_list:
        return None
    
    # Converte a lista de bytes para um número inteiro
    integer_value = int.from_bytes(bytearray(byte_list), byteorder='big', signed=False)
    
    # Converte o inteiro para decimal com precisão e escala
    precision = 15
    scale = 2
    decimal_value = integer_value / (10 ** scale)
    
    # Ajusta o valor para a precisão desejada
    return round(decimal_value, scale)
    

## DEFINE SENSITIVE VARIABLES
WAREHOUSE = os.environ.get("WAREHOUSE") 
AWS_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY_ID") 
AWS_SECRET_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY") 
AWS_S3_ENDPOINT = os.environ.get("AWS_S3_ENDPOINT") ## MINIO ENDPOINT

conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-core_2.12:2.4.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
        .set('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
        .set('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
        .set("spark.hadoop.fs.s3a.path.style.access", "true")
        .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .set("spark.hadoop.fs.s3a.endpoint", "http://minioserver:9000")
        .set('spark.hadoop.fs.s3a.access.key', 'SmRXzZ0KrpROvPSKDddy')
        .set('spark.hadoop.fs.s3a.secret.key', 'ApanXjLP51CxV3bjPqOlankgLDpeoIZyR4xewaOS')
)

# Inicializa a sessão Spark com suporte ao Delta Lake
spark = SparkSession.builder \
    .appName("DataIngestion") \
    .master("local")\
    .config(conf=conf)\
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

def q(query, n=30):
    return spark.sql(query).show(n=n, truncate=False)

def table_exists(namespace, table):
    count = (spark.sql(f'SHOW TABLES FROM {namespace}')
                .filter(f"namespace = '{namespace}' AND tableName = '{table}'")
                .count())
    return count == 1

q('CREATE DATABASE IF NOT EXISTS condomanage')
q('USE condomanage')

# Carrega dados do raw
def load_raw(table_name):
    return spark.read.format('parquet').load(f's3a://condomanage/raw/full/{table_name}')
    
# Salva os dados no MinIO em formato delta
def save_to_delta(df, path):
    df.write.format('delta').mode('overwrite').save(path)

tables = ('condominios', 'moradores', 'transacoes', 'imoveis')
namespace = 'condomanage'

for table in tables:
    if table_exists(namespace, table):
        print(f'Criando tabela {table}...')
        q(f"""
            CREATE  TABLE  IF NOT EXISTS {table}
            USING DELTA
            LOCATION '{WAREHOUSE}/bronze/{table}'
        """)
        df = load_raw(table)
        save_to_delta(df, f"{WAREHOUSE}/bronze/{path}")
    else:
        print(f'Tabela {table} já existente...')

++
||
++
++

++
||
++
++

Criando tabela condominios...
++
||
++
++

Criando tabela moradores...
++
||
++
++

Criando tabela transacoes...
++
||
++
++

Criando tabela imoveis...
++
||
++
++



In [505]:
# AGREGA DADOS CDC moradores

schema = StructType([
    StructField('value', StructType([
        StructField('before', StructType([
            StructField('morador_id', IntegerType(), True), 
            StructField('nome', StringType(), True), 
            StructField('condominio_id', IntegerType(), True), 
            StructField('data_registro', IntegerType(), True)
        ]), True), 
        StructField('after', StructType([
            StructField('morador_id', IntegerType(), True), 
            StructField('nome', StringType(), True), 
            StructField('condominio_id', IntegerType(), True), 
            StructField('data_registro', IntegerType(), True)
        ]), True), 
        StructField('source', StructType([
            StructField('version', StringType(), True), 
            StructField('connector', StringType(), True), 
            StructField('name', StringType(), True), 
            StructField('ts_ms', LongType(), True), 
            StructField('snapshot', StringType(), True), 
            StructField('db', StringType(), True), 
            StructField('sequence', StringType(), True), 
            StructField('schema', StringType(), True), 
            StructField('table', StringType(), True), 
            StructField('txId', LongType(), True), 
            StructField('lsn', LongType(), True), 
            StructField('xmin', LongType(), True)
        ]), True), 
        StructField('op', StringType(), True), 
        StructField('ts_ms', LongType(), True), 
        StructField('transaction', StructType([
            StructField('id', StringType(), True), 
            StructField('total_order', LongType(), True), 
            StructField('data_collection_order', LongType(), True)
        ]), True)
    ]), True)
])

df_stream = spark.readStream.format('parquet')\
    .schema(schema).load(f's3a://condomanage/raw/cdc/postgres.public.moradores')

deltaTable = DeltaTable.forPath(spark, 's3a://condomanage/bronze/moradores')

stream_moradores = (df_stream.writeStream\
    .option("checkpointLocation", f"s3a://condomanage/raw/cdc/moradores_checkpoint/")\
    .foreachBatch(lambda df, batchID: upsert_moradores(df, deltaTable))
    .trigger(availableNow=True))
    


def upsert_moradores(df, deltaTable):
    df_selecionado = df.select(
        "value.after.condominio_id",
        "value.after.data_registro",
        when(df.value.op == 'd', df.value.before.morador_id).otherwise(df.value.after.morador_id).alias('morador_id'),
        "value.after.nome",
        "value.op",
        "value.before",
        "value.ts_ms"
    )
    
   # RENOMEIA AS COLUNAS
    df_selecionado = df_selecionado.withColumnRenamed("value.after.condominio_id", "condominio_id") \
                                   .withColumnRenamed("value.after.data_registro", "data_registro") \
                                   .withColumnRenamed("value.after.morador_id", "morador_id") \
                                   .withColumnRenamed("value.after.nome", "nome") \
                                   .withColumnRenamed("value.op", "op") \
                                   .withColumnRenamed("value.before", "before") \
                                   .withColumnRenamed("value.ts_ms", "ts_ms")
    
    # SELECIONA A ULTIMA ATUALIZAÇÃO DO DADO
    df_selecionado.createOrReplaceGlobalTempView('view_moradores')
    
    cdc_unique = spark.sql('''
    WITH 
        qualify as (select 
            *, 
            ROW_NUMBER() over(partition by morador_id order by ts_ms desc) as qualify 
        from global_temp.view_moradores)
    select * from qualify where qualify = 1''')
    
    # AJUSTA O TIPO DA COLUNA DATA_REGISTRO
    cdc_unique = cdc_unique.withColumn("data_registro", to_date(expr("date_add('1970-01-01', data_registro)")))
    
    # UPSERT
    (deltaTable.alias('b')
        .merge(cdc_unique.alias('d'), 
        'b.morador_id = d.morador_id')
        .whenMatchedUpdateAll(condition = "d.op = 'u'")
        .whenMatchedDelete(condition = "d.op = 'd'")
        .whenNotMatchedInsertAll(condition = "d.op = 'c' or d.op = 'u'")
        .execute()
    )


In [506]:
stream_moradores.start()

In [508]:
q('select * from moradores order by 1 desc')

+----------+------------------+-------------+-------------+
|morador_id|nome              |condominio_id|data_registro|
+----------+------------------+-------------+-------------+
|604       |Tiago             |642          |2024-08-01   |
|603       |Andre             |643          |2024-08-01   |
|602       |Amber Bishop      |682          |2024-05-28   |
|601       |Laura Guerrero    |603          |2023-05-24   |
|600       |Christopher Petty |659          |2023-12-03   |
|599       |Jonathan Page     |683          |2023-07-20   |
|598       |Donald Barrett    |673          |2020-08-20   |
|597       |Russell Carr      |605          |2024-06-19   |
|596       |Michele Dawson    |639          |2023-04-08   |
|595       |Nicole Mack       |689          |2021-12-27   |
|594       |Angelica Krueger  |674          |2022-06-28   |
|593       |Donna Wells       |663          |2023-12-19   |
|592       |Katherine Martinez|658          |2024-02-17   |
|591       |Elizabeth Cummings|637      

In [442]:
# AGREGA CDC CONDOMINIOS
from pyspark.sql.types import DecimalType

schema = StructType([
    StructField('value', StructType([
        StructField('before', StructType([
            StructField('condominio_id', IntegerType(), True), 
            StructField('nome', StringType(), True), 
            StructField('endereco', StringType(), True)
        ]), True), 
        StructField('after', 
            StructType([
                StructField('condominio_id', IntegerType(), True), 
                StructField('nome', StringType(), True), 
                StructField('endereco', StringType(), True)
            ]), True), 
        StructField('source', 
                    StructType([
                        StructField('version', StringType(), True), 
                        StructField('connector', StringType(), True), 
                        StructField('name', StringType(), True), 
                        StructField('ts_ms', LongType(), True), 
                        StructField('snapshot', StringType(), True),
                        StructField('db', StringType(), True), 
                        StructField('sequence', StringType(), True), 
                        StructField('schema', StringType(), True), 
                        StructField('table', StringType(), True), 
                        StructField('txId', LongType(), True), 
                        StructField('lsn', LongType(), True), 
                        StructField('xmin', LongType(), True)
                    ]), True), 
        StructField('op', StringType(), True), 
        StructField('ts_ms', LongType(), True), 
        StructField('transaction', StructType([
            StructField('id', StringType(), True), 
            StructField('total_order', LongType(), True), 
            StructField('data_collection_order', LongType(), True)
        ]), True)
    ]), True)
])

df_stream = spark.readStream.format('parquet')\
    .schema(schema).load(f's3a://condomanage/raw/cdc/postgres.public.condominios')

deltaTable = DeltaTable.forPath(spark, 's3a://condomanage/bronze/condominios')

stream_condominios = (df_stream.writeStream\
    .option("checkpointLocation", f"s3a://condomanage/raw/cdc/condominios_checkpoint/")\
    .foreachBatch(lambda df, batchID: upsert_condominios(df, deltaTable))
    .trigger(availableNow=True))
    


def upsert_condominios(df, deltaTable):
    df_selecionado = df.select(
        "value.after.endereco",
        "value.after.nome",
        when(df.value.op == 'd', df.value.before.condominio_id).otherwise(df.value.after.condominio_id).alias('condominio_id'),
        "value.op",
        "value.before",
        "value.ts_ms"
    )
    
   # RENOMEIA AS COLUNAS
    df_selecionado = df_selecionado.withColumnRenamed("value.after.endereco", "endereco") \
                                   .withColumnRenamed("value.after.nome", "nome") \
                                   .withColumnRenamed("value.after.condominio_id", "condominio_id") \
                                   .withColumnRenamed("value.op", "op") \
                                   .withColumnRenamed("value.before", "before") \
                                   .withColumnRenamed("value.ts_ms", "ts_ms")
    
    # SELECIONA A ULTIMA ATUALIZAÇÃO DO DADO
    df_selecionado.createOrReplaceGlobalTempView('view_condominios')
    
    cdc_unique = spark.sql('''
    WITH 
        qualify as (select 
            *, 
            ROW_NUMBER() over(partition by condominio_id order by ts_ms desc) as qualify 
        from global_temp.view_condominios)
    select * from qualify where qualify = 1''')
    
    # UPSERT
    (deltaTable.alias('b')
        .merge(cdc_unique.alias('d'), 
        'b.condominio_id = d.condominio_id')
        .whenMatchedUpdateAll(condition = "d.op = 'u'")
        .whenMatchedDelete(condition = "d.op = 'd'")
        .whenNotMatchedInsertAll(condition = "d.op = 'c'")
        .execute()
    )
   

In [ ]:
# IMOVEIS Stream
# Definindo o schema do PySpark
schema = StructType([
StructField('value', StructType([
    StructField("before", StructType([
        StructField("imovel_id", IntegerType(), nullable=False),
        StructField("tipo", StringType(), nullable=False),
        StructField("condominio_id", IntegerType(), nullable=False),
        StructField("valor", BinaryType(), nullable=False)  # Coluna binária para decimal
    ]), nullable=True),
    StructField("after", StructType([
        StructField("imovel_id", IntegerType(), nullable=False),
        StructField("tipo", StringType(), nullable=False),
        StructField("condominio_id", IntegerType(), nullable=False),
        StructField("valor", BinaryType(), nullable=False)  # Coluna binária para decimal
    ]), nullable=True),
    StructField("source", StructType([
        StructField("version", StringType(), nullable=False),
        StructField("connector", StringType(), nullable=False),
        StructField("name", StringType(), nullable=False),
        StructField("ts_ms", LongType(), nullable=False),
        StructField("snapshot", StringType(), nullable=True),
        StructField("db", StringType(), nullable=False),
        StructField("sequence", StringType(), nullable=True),
        StructField("schema", StringType(), nullable=False),
        StructField("table", StringType(), nullable=False),
        StructField("txId", LongType(), nullable=True),
        StructField("lsn", LongType(), nullable=True),
        StructField("xmin", LongType(), nullable=True)
    ]), nullable=False),
    StructField("op", StringType(), nullable=False),
    StructField("ts_ms", LongType(), nullable=True),
    StructField("transaction", StructType([
        StructField("id", StringType(), nullable=False),
        StructField("total_order", LongType(), nullable=False),
        StructField("data_collection_order", LongType(), nullable=False)
    ]), nullable=True)
]))
])

df_stream = spark.readStream.format('parquet')\
    .schema(schema).load(f's3a://condomanage/raw/cdc/postgres.public.imoveis')

deltaTable = DeltaTable.forPath(spark, 's3a://condomanage/bronze/imoveis')

stream = (df_stream.writeStream\
    .option("checkpointLocation", f"s3a://condomanage/raw/cdc/imoveis_checkpoint/")\
    .foreachBatch(lambda df, batchID: upsert_imoveis(df, deltaTable))
    .trigger(availableNow=True))
    


def upsert_imoveis(df, deltaTable):
    df_selecionado = df.select(
        "value.after.tipo",
        "value.after.valor",
        "value.after.condominio_id",
        when(df.value.op == 'd', df.value.before.imovel_id).otherwise(df.value.after.imovel_id).alias('imovel_id'),
        "value.op",
        #"value.before",
        "value.ts_ms"
    )
    
    df_selecionado = df_selecionado.withColumnRenamed("value.after.tipo", "tipo") \
                                   .withColumnRenamed("value.after.valor", "valor") \
                                   .withColumnRenamed("value.after.condominio_id", "condominio_id") \
                                    .withColumnRenamed("value.after.imovel_id", "imovel_id") \
                                   .withColumnRenamed("value.op", "op") \
                                   .withColumnRenamed("value.ts_ms", "ts_ms")
    df =df_selecionado.toPandas()
    
    df['valor'] = df.apply(lambda x: bytes_to_decimal(x['valor']), axis=1) 
    
    schema = StructType([
        StructField("tipo", StringType(), nullable=False),
        StructField("valor", DecimalType(15, 2), nullable=False),
        StructField("condominio_id", IntegerType(), nullable=False),
        StructField("imovel_id", IntegerType(), nullable=False),
        StructField("condominio_id", IntegerType(), nullable=False),
        StructField("op", StringType(), nullable=False),
        StructField("ts_ms", LongType(), nullable=True)
    ])
    
    df_selecionado = spark.createDataFrame(df)
    
    # SELECIONA A ULTIMA ATUALIZAÇÃO DO DADO
    df_selecionado.createOrReplaceGlobalTempView('view_imoveis')
    
    cdc_unique = spark.sql('''
    WITH 
        qualify as (select 
            *, 
            ROW_NUMBER() over(partition by imovel_id order by ts_ms desc) as qualify 
        from global_temp.view_imoveis)
    select * from qualify where qualify = 1''')
    
    # UPSERT
    (deltaTable.alias('b')
        .merge(cdc_unique.alias('d'), 
        'b.imovel_id = d.imovel_id')
        .whenMatchedUpdateAll(condition = "d.op = 'u'")
        .whenMatchedDelete(condition = "d.op = 'd'")
        .whenNotMatchedInsertAll(condition = "d.op = 'c'")
        .execute()
    )

In [224]:
# AGREGA CDC TRANSACOES
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType, DecimalType, BinaryType
import pandas as pd
import struct

# Função para converter bytes para decimal
def bytes_to_decimal(byte_list):
    # Converte a lista de bytes para um número inteiro
    integer_value = int.from_bytes(bytearray(byte_list), byteorder='big', signed=False)
    
    # Converte o inteiro para decimal com precisão e escala
    precision = 15
    scale = 2
    decimal_value = integer_value / (10 ** scale)
    
    # Ajusta o valor para a precisão desejada
    return round(decimal_value, scale)

# Definindo o schema do PySpark
schema = StructType([
StructField('value', StructType([
    StructField("before", StructType([
        StructField("transacao_id", IntegerType(), nullable=False),
        StructField("imovel_id", IntegerType(), nullable=False),
        StructField("morador_id", IntegerType(), nullable=False),
        StructField("data_transacao", IntegerType(), nullable=False),
        StructField("valor_transacao", BinaryType(), nullable=False),# Coluna binária para decimal
    ]), nullable=True),
    StructField("after", StructType([
        StructField("transacao_id", IntegerType(), nullable=False),
        StructField("imovel_id", IntegerType(), nullable=False),
        StructField("morador_id", IntegerType(), nullable=False),
        StructField("data_transacao", IntegerType(), nullable=False),
        StructField("valor_transacao", BinaryType(), nullable=False),# Coluna binária para decimal
    ]), nullable=True),
    StructField("source", StructType([
        StructField("version", StringType(), nullable=False),
        StructField("connector", StringType(), nullable=False),
        StructField("name", StringType(), nullable=False),
        StructField("ts_ms", LongType(), nullable=False),
        StructField("snapshot", StringType(), nullable=True),
        StructField("db", StringType(), nullable=False),
        StructField("sequence", StringType(), nullable=True),
        StructField("schema", StringType(), nullable=False),
        StructField("table", StringType(), nullable=False),
        StructField("txId", LongType(), nullable=True),
        StructField("lsn", LongType(), nullable=True),
        StructField("xmin", LongType(), nullable=True)
    ]), nullable=False),
    StructField("op", StringType(), nullable=False),
    StructField("ts_ms", LongType(), nullable=True),
    StructField("transaction", StructType([
        StructField("id", StringType(), nullable=False),
        StructField("total_order", LongType(), nullable=False),
        StructField("data_collection_order", LongType(), nullable=False)
    ]), nullable=True)
]))
            ])
df = spark.read.format("parquet").schema(schema).load('s3a://condomanage/raw/cdc/postgres.public.transacoes')

df_selecionado = df.select(
    "value.after.imovel_id",
    "value.after.morador_id",
    "value.after.data_transacao",
    "value.after.valor_transacao",
    when(df.value.op == 'd', df.value.before.transacao_id).otherwise(df.value.after.transacao_id).alias('transacao_id'),
    "value.op",
    "value.ts_ms"
)

df_selecionado = df_selecionado.withColumnRenamed("value.after.imovel_id", "imovel_id") \
                               .withColumnRenamed("value.after.morador_id", "morador_id") \
                               .withColumnRenamed("value.after.data_transacao", "data_transacao") \
                               .withColumnRenamed("value.after.valor_transacao", "valor_transacao") \
                               .withColumnRenamed("value.after.transacao_id", "transacao_id") \
                               .withColumnRenamed("value.op", "op") \
                               .withColumnRenamed("value.ts_ms", "ts_ms")
df =df_selecionado.toPandas()

df['valor_transacao'] = df.apply(lambda x: bytes_to_decimal(x['valor_transacao']), axis=1) 

schema = StructType([
    StructField("imovel_id", IntegerType(), nullable=False),
    StructField("morador_id", IntegerType(), nullable=False),
    StructField("data_transacao", IntegerType(), nullable=False),
    StructField("valor_transacao", DecimalType(15, 2), nullable=False),
    StructField("transacao_id", IntegerType(), nullable=False),
    StructField("op", StringType(), nullable=False),
    StructField("ts_ms", LongType(), nullable=True)
])

df_selecionado = spark.createDataFrame(df)

# SELECIONA A ULTIMA ATUALIZAÇÃO DO DADO
df_selecionado.createOrReplaceTempView('delta_t')

cdc_unique = spark.sql('''
WITH 
    qualify as (select 
        imovel_id, morador_id, cast(data_transacao as integer), valor_transacao, transacao_id, op, ts_ms,
        ROW_NUMBER() over(partition by transacao_id order by ts_ms desc) as qualify 
    from delta_t)
select * from qualify where qualify = 1''')

# AJUSTA O TIPO DA COLUNA DATA_REGISTRO
cdc_unique = cdc_unique.withColumn("data_transacao", to_date(expr("date_add('1970-01-01', data_transacao)")))

bronze = DeltaTable.forPath(spark, 's3a://condomanage/bronze/transacoes')

# UPSERT
(bronze.alias('b')
    .merge(cdc_unique.alias('d'), 
    'b.transacao_id = d.transacao_id')
    .whenMatchedUpdateAll(condition = "d.op = 'u'")
    .whenMatchedDelete(condition = "d.op = 'd'")
    .whenNotMatchedInsertAll(condition = "d.op = 'c'")
    .execute()
)

In [217]:
q('select * from transacoes where imovel_id = 502')


+------------+---------+----------+--------------+---------------+
|transacao_id|imovel_id|morador_id|data_transacao|valor_transacao|
+------------+---------+----------+--------------+---------------+
|550         |502      |566       |2024-05-01    |221267.96      |
|597         |502      |597       |2024-06-05    |167102.95      |
+------------+---------+----------+--------------+---------------+



In [225]:
q('select * from transacoes where imovel_id = 502')



+------------+---------+----------+--------------+---------------+
|transacao_id|imovel_id|morador_id|data_transacao|valor_transacao|
+------------+---------+----------+--------------+---------------+
|550         |502      |566       |2024-05-01    |1.00           |
|597         |502      |597       |2024-06-03    |167102.95      |
|601         |502      |566       |2024-08-01    |100000.00      |
+------------+---------+----------+--------------+---------------+



In [164]:
q(f"""
            CREATE  TABLE  IF NOT EXISTS teste
            USING DELTA
            LOCATION 's3a://condomanage/teste'
        """)

++
||
++
++



In [183]:
bytes_to_decimal_udf = udf(bytes_to_decimal, DecimalType(15, 2))


In [438]:
stream.start()


In [441]:
q('select * from condominios where condominio_id = 620')



+-------------+-----------------------------+---------------------------------------------------------+
|condominio_id|nome                         |endereco                                                 |
+-------------+-----------------------------+---------------------------------------------------------+
|620          |Page, Griffin and Vazquez vcc|866 Brandon Underpass Suite 051, Kristinborough, NY 53179|
+-------------+-----------------------------+---------------------------------------------------------+



In [408]:
dir(stream)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_df',
 '_jwrite',
 '_spark',
 '_sq',
 'foreach',
 'foreachBatch',
 'format',
 'option',
 'options',
 'outputMode',
 'partitionBy',
 'queryName',
 'start',
 'toTable',
 'trigger']